In [16]:
from bs4 import BeautifulSoup
import requests
import re

link = 'https://coinmarketcap.com/pt-br/'
req = requests.get(link)
site = BeautifulSoup(req.text, 'html.parser')

#### Extraindo o nome das 10 primeiras moedas na tabela

In [17]:
tabela = site.find('tbody')
moedas_classe = tabela.find_all(class_='ehyBa-d')
moedas_nome = [moeda.contents[0] for moeda in moedas_classe]
moedas_nome

['Bitcoin',
 'Ethereum',
 'Tether',
 'BNB',
 'Solana',
 'USDC',
 'XRP',
 'Toncoin',
 'Dogecoin',
 'Cardano']

#### Extraindo o preço das moedas

In [18]:
preco_classe = tabela.find_all(class_='gPTgRa')
precos = [preco.contents[0].text for preco in preco_classe]
precos

['R$340,249.93',
 'R$18,649.29',
 'R$5.50',
 'R$3,169.76',
 'R$765.13',
 'R$5.50',
 'R$2.62',
 'R$41.21',
 'R$0.6891',
 'R$2.17']

#### Extraindo a informação de variação das moedas

Aqui as variações estão distribuídas da seguinte forma: O primeiro item é a variação de 1h, o segundo é a variação de 24h e o terceiro é a variação de 7 dias. Os três primeiros itens da lista são as variações da primeria moeda da tabela, os próximos 3 são as variaçoes da segunda moeda e assim vai.

In [19]:
variacoes = tabela.find_all(string=re.compile("%"))
for i, variacao in enumerate(variacoes):
  if "ivvJzO" in variacao.parent['class']:
    variacoes[i] = "-" + variacao
variacoes

['0.57%',
 '2.18%',
 '-4.31%',
 '0.44%',
 '1.90%',
 '-3.42%',
 '-0.19%',
 '1.31%',
 '0.68%',
 '0.61%',
 '1.87%',
 '-2.79%',
 '1.41%',
 '2.90%',
 '2.09%',
 '-0.19%',
 '1.27%',
 '0.66%',
 '0.15%',
 '1.24%',
 '-2.80%',
 '0.22%',
 '-0.02%',
 '9.03%',
 '0.98%',
 '2.51%',
 '1.94%',
 '0.72%',
 '2.10%',
 '2.62%']

#### Extraindo a informação de capitalização de mercado para cada moeda

A capitalização de mercado é:<br> O valor total de mercado do fornecimento em circulação de uma criptomoeda. É semelhante à capitalização do free float no mercado de ações.

Capitalizado de Mercado = Preço Atual x Fornecimento em Circulação.

In [20]:
cap_elementos = tabela.find_all(class_='hwOFkt')
cap_mercado = [cap.contents[0] for cap in cap_elementos]
cap_mercado

['R$6,709,090,838,122',
 'R$2,241,609,032,598',
 'R$620,564,721,744',
 'R$467,629,526,918',
 'R$353,678,195,026',
 'R$181,054,711,920',
 'R$145,527,132,577',
 'R$101,299,287,836',
 'R$99,823,129,922',
 'R$77,458,897,999']

#### Extraindo o volume de cada moeda

O volume é uma medida de quanto de uma criptomoeda foi negociada nas últimas 24 horas.

In [21]:
volume_elementos = tabela.find_all(class_='bbHOdE')
volume_24h = [volume.contents[0].text for volume in volume_elementos]
volume_24h

['R$6.71T',
 'R$139,736,146,182',
 'R$2.24T',
 'R$63,794,824,970',
 'R$620.56B',
 'R$261,422,502,220',
 'R$467.63B',
 'R$8,440,589,877',
 'R$353.68B',
 'R$10,758,562,133',
 'R$181.05B',
 'R$29,582,545,461',
 'R$145.53B',
 'R$4,824,518,462',
 'R$101.3B',
 'R$1,470,308,948',
 'R$99.82B',
 'R$3,355,089,523',
 'R$77.46B',
 'R$1,315,282,728']

#### Existem outros elementos no site com a mesma classe que o volume, na seguinte célula eu trato esse problema, garantindo que estou extraindo apenas o volume.

In [22]:
for volume in volume_24h:
  if volume.endswith('T') or volume.endswith('B'):
    volume_24h.remove(volume)
volume_24h

['R$139,736,146,182',
 'R$63,794,824,970',
 'R$261,422,502,220',
 'R$8,440,589,877',
 'R$10,758,562,133',
 'R$29,582,545,461',
 'R$4,824,518,462',
 'R$1,470,308,948',
 'R$3,355,089,523',
 'R$1,315,282,728']

#### Extraindo o fornecimento circulante de cada moeda

O fornecimento circulante é a quantidade de moedas que estão circulando no mercado e estão nas mãos do público. É parecido com a flutuação de ações no mercado de ações.

In [23]:
fornecimento_elementos = tabela.find_all(class_='hhmVNu')
fornecimento_circulante = [each.text for each in fornecimento_elementos]
fornecimento_circulante

['19,717,071 BTC',
 '120,178,989 ETH',
 '112,829,626,104 USDT',
 '147,583,445 BNB',
 '462,246,430 SOL',
 '32,912,286,253 USDC',
 '55,618,185,850 XRP',
 '2,458,373,537 TON',
 '144,868,946,384 DOGE',
 '35,745,472,248 ADA']

## Criando um dicionário com as informações de cada moeda

In [24]:
moedas = {moeda: {"Preço": precos[i]} for i, moeda in enumerate(moedas_nome)}

for i, moeda in enumerate(moedas):
  moedas[moeda]['Variação 1h'] = variacoes.pop(i-i)
  moedas[moeda]['Variação 24h'] = variacoes.pop(i-i)
  moedas[moeda]['Variação 7d'] = variacoes.pop(i-i)

for i, moeda in enumerate(moedas):
  moedas[moeda]['Cap. de Mercado'] = cap_mercado[i]

for i, moeda in enumerate(moedas):
  moedas[moeda]['Volume (24h)'] = volume_24h[i]

for i, moeda in enumerate(moedas):
  moedas[moeda]['Fornecimento Circulante'] = fornecimento_circulante[i]

for moeda in moedas:
  print(moeda, moedas[moeda])
  print()

Bitcoin {'Preço': 'R$340,249.93', 'Variação 1h': '0.57%', 'Variação 24h': '2.18%', 'Variação 7d': '-4.31%', 'Cap. de Mercado': 'R$6,709,090,838,122', 'Volume (24h)': 'R$139,736,146,182', 'Fornecimento Circulante': '19,717,071 BTC'}

Ethereum {'Preço': 'R$18,649.29', 'Variação 1h': '0.44%', 'Variação 24h': '1.90%', 'Variação 7d': '-3.42%', 'Cap. de Mercado': 'R$2,241,609,032,598', 'Volume (24h)': 'R$63,794,824,970', 'Fornecimento Circulante': '120,178,989 ETH'}

Tether {'Preço': 'R$5.50', 'Variação 1h': '-0.19%', 'Variação 24h': '1.31%', 'Variação 7d': '0.68%', 'Cap. de Mercado': 'R$620,564,721,744', 'Volume (24h)': 'R$261,422,502,220', 'Fornecimento Circulante': '112,829,626,104 USDT'}

BNB {'Preço': 'R$3,169.76', 'Variação 1h': '0.61%', 'Variação 24h': '1.87%', 'Variação 7d': '-2.79%', 'Cap. de Mercado': 'R$467,629,526,918', 'Volume (24h)': 'R$8,440,589,877', 'Fornecimento Circulante': '147,583,445 BNB'}

Solana {'Preço': 'R$765.13', 'Variação 1h': '1.41%', 'Variação 24h': '2.90%', 'V

## Criando o dataframe

In [25]:
import pandas as pd

moedas_df = pd.DataFrame(moedas)
moedas_df

,Bitcoin,Ethereum,Tether,BNB,Solana,USDC,XRP,Toncoin,Dogecoin,Cardano
Preço,"R$340,249.93","R$18,649.29",R$5.50,"R$3,169.76",R$765.13,R$5.50,R$2.62,R$41.21,R$0.6891,R$2.17
Variação 1h,0.57%,0.44%,-0.19%,0.61%,1.41%,-0.19%,0.15%,0.22%,0.98%,0.72%
Variação 24h,2.18%,1.90%,1.31%,1.87%,2.90%,1.27%,1.24%,-0.02%,2.51%,2.10%
Variação 7d,-4.31%,-3.42%,0.68%,-2.79%,2.09%,0.66%,-2.80%,9.03%,1.94%,2.62%
Cap. de Mercado,"R$6,709,090,838,122","R$2,241,609,032,598","R$620,564,721,744","R$467,629,526,918","R$353,678,195,026","R$181,054,711,920","R$145,527,132,577","R$101,299,287,836","R$99,823,129,922","R$77,458,897,999"
Volume (24h),"R$139,736,146,182","R$63,794,824,970","R$261,422,502,220","R$8,440,589,877","R$10,758,562,133","R$29,582,545,461","R$4,824,518,462","R$1,470,308,948","R$3,355,089,523","R$1,315,282,728"
Fornecimento Circulante,"19,717,071 BTC","120,178,989 ETH","112,829,626,104 USDT","147,583,445 BNB","462,246,430 SOL","32,912,286,253 USDC","55,618,185,850 XRP","2,458,373,537 TON","144,868,946,384 DOGE","35,745,472,248 ADA"


### Criando uma função para consultar uma moeda específica

In [26]:
def consultar_moeda():
  moeda = input("Qual moeda você deseja consultar?")
  moeda = moeda.casefold()
  moeda = moeda.title()
  if moeda == 'Bnb':
    moeda = moeda.upper()
  elif moeda == 'Usdc':
    moeda = moeda.upper()
  elif moeda == 'Xrp':
    moeda = moeda.upper()
  def consultar(moeda):
    try:
      print(moeda)
      return moedas[moeda]
    except:
      print("Moeda inválida!")
  return consultar(moeda)

consultar_moeda()

Solana


{'Preço': 'R$765.13',
 'Variação 1h': '1.41%',
 'Variação 24h': '2.90%',
 'Variação 7d': '2.09%',
 'Cap. de Mercado': 'R$353,678,195,026',
 'Volume (24h)': 'R$10,758,562,133',
 'Fornecimento Circulante': '462,246,430 SOL'}